In [1]:
# !pip install -qU "langchain[google-vertexai]"

In [2]:
from typing import Optional, List
from pydantic import BaseModel, Field
from enum import Enum
from model import ResumeFields
from langchain.chat_models import init_chat_model

In [3]:
f = open("./data/linkedin_cv.txt", "r")
linkedin_target_resume = f.read()

In [4]:
llm = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai", temperature = 0.3)

In [5]:
system_prompt = """
You are a highly skilled AI assistant specializing in analyzing job descriptions and extracting key requirements. Your goal is to meticulously identify and categorize the essential skills, qualifications, experience, and other attributes sought by employers.

**Your Task:**

Given a job description as input, extract the required fields based on schema into a structured format. Be comprehensive and specific, avoiding vague or generic terms.

**Instructions:**

1.  **Read the job description carefully and thoroughly.**
2.  **Focus on explicitly stated requirements.**  Infer requirements only when strongly implied and directly supported by the text.
3.  **Be specific and detailed.** Avoid generalizations. For example, instead of "programming skills," specify "Proficiency in Python and Java."
4.  **Prioritize "must-have" requirements.**  Distinguish between essential requirements and desired or "nice-to-have" qualifications if the job description makes that distinction. 
5.  **Maintain accuracy.**  Do not add information that is not present in the job description.
6.  **Conciseness is important,** but don't sacrifice accuracy or completeness.
"""

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt
        ),
        ("human", "{text}"),
    ]
)

In [7]:
runnable = prompt | llm.with_structured_output(
    schema=ResumeFields
)

fields = runnable.invoke({"text": (
    f"{linkedin_target_resume}"
    "Extract all the relevant fields from the text given"
)})

Key '$defs' is not supported in schema, ignoring


In [8]:
from rich import print 
print(fields)

ResumeFields(
    skills=[
        Skill(skill='Java', skill_type=<SkillType.TECHNICAL: 'technical'>, experience=None, is_required=True),
        Skill(skill='Python', skill_type=<SkillType.TECHNICAL: 'technical'>, experience=None, is_required=True),
        Skill(skill='C++', skill_type=<SkillType.TECHNICAL: 'technical'>, experience=None, is_required=True),
        Skill(
            skill='scripting',
            skill_type=<SkillType.TECHNICAL: 'technical'>,
            experience=None,
            is_required=True
        ),
        Skill(
            skill='test automation',
            skill_type=<SkillType.TECHNICAL: 'technical'>,
            experience=None,
            is_required=True
        ),
        Skill(skill='HTML5', skill_type=<SkillType.TECHNICAL: 'technical'>, experience=None, is_required=False),
        Skill(skill='CSS3', skill_type=<SkillType.TECHNICAL: 'technical'>, experience=None, is_required=False),
        Skill(skill='SQL', skill_type=<SkillType.TECHNICAL: 'technical'>, experience=None, is_required=False),
        Skill(skill='ORM', skill_type=<SkillType.TECHNICAL: 'technical'>, experience=None, is_required=False),
        Skill(skill='Wicket', skill_type=<SkillType.TECHNICAL: 'technical'>, experience=None, is_required=False),
        Skill(skill='GWT', skill_type=<SkillType.TECHNICAL: 'technical'>, experience=None, is_required=False),
        Skill(
            skill='Spring MVC',
            skill_type=<SkillType.TECHNICAL: 'technical'>,
            experience=None,
            is_required=False
        )
    ],
    experiences=[
        Experience(
            experience='Five or more years of experience as engineer of software and networking platforms',
            is_required=True
        ),
        Experience(
            experience='Seven or more years of experience (professional and academic) with Java, Python, and C++',
            is_required=True
        ),
        Experience(
            experience='Experience with rapid development cycles in a web-based environment',
            is_required=True
        )
    ],
    qualifications=[
        Qualification(
            degree='Bachelor’s degree (or equivalent) in software engineering or information technology',
            is_required=False
        )
    ],
    personality_traits=[]
)

In [9]:
import pickle

with open('./outputs/linkedin_cv.pkl', 'wb') as f:
    pickle.dump(fields, f)

In [10]:
# with open('./outputs/linkedin_cv.pkl', 'rb') as f:
#     tmp = pickle.load(f)